In [10]:
import torch
import torchvision
from torchvision.transforms import ToTensor, Normalize, Compose
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
import torch.nn as nn

from IPython.display import Image
from torchvision.utils import save_image

mnist = MNIST(root='./../data', 
              train=True, 
              download=True,
              transform=Compose([ToTensor(), Normalize(mean=(0.5,), std=(0.5,))]))

In [11]:
image_size = 784
hidden_size = 256
latent_size = 64

batch_size = 100
data_loader = DataLoader(mnist, batch_size, shuffle=True)

In [12]:
D = nn.Sequential(
    nn.Linear(image_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, 1),
    nn.Sigmoid())

G = nn.Sequential(
    nn.Linear(latent_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, image_size),
    nn.Tanh())

In [13]:
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0002)
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0002)

In [14]:
def reset_grad():
    d_optimizer.zero_grad()
    g_optimizer.zero_grad()

def train_discriminator(images):
    real_labels = torch.ones(batch_size, 1)
    fake_labels = torch.zeros(batch_size, 1)
        
    outputs = D(images)
    d_loss_real = criterion(outputs, real_labels)
    real_score = outputs

    z = torch.randn(batch_size, latent_size)
    fake_images = G(z)
    outputs = D(fake_images)
    d_loss_fake = criterion(outputs, fake_labels)
    fake_score = outputs

    d_loss = d_loss_real + d_loss_fake
    reset_grad()
    d_loss.backward()
    d_optimizer.step()
    
    return d_loss, real_score, fake_score

In [15]:
def train_generator():
    z = torch.randn(batch_size, latent_size)
    fake_images = G(z)
    labels = torch.ones(batch_size, 1)
    g_loss = criterion(D(fake_images), labels)

    reset_grad()
    g_loss.backward()
    g_optimizer.step()
    return g_loss, fake_images

In [16]:
import os

sample_dir = './../data/samples'
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

In [20]:
sample_vectors = torch.randn(batch_size, latent_size)

def save_fake_images(index):
    fake_images = G(sample_vectors)
    fake_images = fake_images.reshape(fake_images.size(0), 1, 28, 28)
    fake_fname = 'pre-train_images-{0:0=4d}.png'.format(index)
    print('Saving', fake_fname)
    save_image(fake_images, os.path.join(sample_dir, fake_fname), nrow=10)

In [26]:
%%time

num_epochs = 50
total_step = len(data_loader)
d_losses, g_losses, real_scores, fake_scores = [], [], [], []

for epoch in range(num_epochs):
    for i, (images, _) in enumerate(data_loader):
        # Load a batch & transform to vectors
        images = images.reshape(batch_size, -1)
        
        # Train the discriminator and generator
        d_loss, real_score, fake_score = train_discriminator(images)
        #g_loss, fake_images = train_generator()
        
        if (i+1) % 200 == 0:
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}' 
                  .format(epoch, num_epochs, i+1, total_step, d_loss.item(), g_loss.item(), 
                          real_score.mean().item(), fake_score.mean().item()))

Epoch [0/50], Step [200/600], d_loss: 0.0000, g_loss: 0.0000, D(x): 1.00, D(G(z)): 0.00
Epoch [0/50], Step [400/600], d_loss: 0.0000, g_loss: 0.0000, D(x): 1.00, D(G(z)): 0.00
Epoch [0/50], Step [600/600], d_loss: 0.0000, g_loss: 0.0000, D(x): 1.00, D(G(z)): 0.00
Epoch [1/50], Step [200/600], d_loss: 0.0000, g_loss: 0.0000, D(x): 1.00, D(G(z)): 0.00
Epoch [1/50], Step [400/600], d_loss: 0.0000, g_loss: 0.0000, D(x): 1.00, D(G(z)): 0.00
Epoch [1/50], Step [600/600], d_loss: 0.0000, g_loss: 0.0000, D(x): 1.00, D(G(z)): 0.00
Epoch [2/50], Step [200/600], d_loss: 0.0000, g_loss: 0.0000, D(x): 1.00, D(G(z)): 0.00
Epoch [2/50], Step [400/600], d_loss: 0.0000, g_loss: 0.0000, D(x): 1.00, D(G(z)): 0.00
Epoch [2/50], Step [600/600], d_loss: 0.0000, g_loss: 0.0000, D(x): 1.00, D(G(z)): 0.00
Epoch [3/50], Step [200/600], d_loss: 0.0000, g_loss: 0.0000, D(x): 1.00, D(G(z)): 0.00
Epoch [3/50], Step [400/600], d_loss: 0.0000, g_loss: 0.0000, D(x): 1.00, D(G(z)): 0.00
Epoch [3/50], Step [600/600], d_

Epoch [31/50], Step [200/600], d_loss: 0.0000, g_loss: 0.0000, D(x): 1.00, D(G(z)): 0.00
Epoch [31/50], Step [400/600], d_loss: 0.0000, g_loss: 0.0000, D(x): 1.00, D(G(z)): 0.00
Epoch [31/50], Step [600/600], d_loss: 0.0000, g_loss: 0.0000, D(x): 1.00, D(G(z)): 0.00
Epoch [32/50], Step [200/600], d_loss: 0.0000, g_loss: 0.0000, D(x): 1.00, D(G(z)): 0.00
Epoch [32/50], Step [400/600], d_loss: 0.0000, g_loss: 0.0000, D(x): 1.00, D(G(z)): 0.00
Epoch [32/50], Step [600/600], d_loss: 0.0000, g_loss: 0.0000, D(x): 1.00, D(G(z)): 0.00
Epoch [33/50], Step [200/600], d_loss: 0.0000, g_loss: 0.0000, D(x): 1.00, D(G(z)): 0.00
Epoch [33/50], Step [400/600], d_loss: 0.0000, g_loss: 0.0000, D(x): 1.00, D(G(z)): 0.00
Epoch [33/50], Step [600/600], d_loss: 0.0000, g_loss: 0.0000, D(x): 1.00, D(G(z)): 0.00
Epoch [34/50], Step [200/600], d_loss: 0.0000, g_loss: 0.0000, D(x): 1.00, D(G(z)): 0.00
Epoch [34/50], Step [400/600], d_loss: 0.0000, g_loss: 0.0000, D(x): 1.00, D(G(z)): 0.00
Epoch [34/50], Step [

In [ ]:
for epoch in range(num_epochs):
    for i, (images, _) in enumerate(data_loader):
        g_loss, fake_images = train_generator()
        
        if (i+1) % 200 == 0:
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}' 
                  .format(epoch, num_epochs, i+1, total_step, d_loss.item(), g_loss.item(), 
                          real_score.mean().item(), fake_score.mean().item()))
        
    # Sample and save images
    save_fake_images(epoch+1)

Epoch [0/50], Step [200/600], d_loss: 0.0000, g_loss: 0.0000, D(x): 1.00, D(G(z)): 0.00
Epoch [0/50], Step [400/600], d_loss: 0.0000, g_loss: 0.0000, D(x): 1.00, D(G(z)): 0.00
Epoch [0/50], Step [600/600], d_loss: 0.0000, g_loss: 0.0000, D(x): 1.00, D(G(z)): 0.00
Saving pre-train_images-0001.png
Epoch [1/50], Step [200/600], d_loss: 0.0000, g_loss: 0.0000, D(x): 1.00, D(G(z)): 0.00
Epoch [1/50], Step [400/600], d_loss: 0.0000, g_loss: 0.0000, D(x): 1.00, D(G(z)): 0.00
Epoch [1/50], Step [600/600], d_loss: 0.0000, g_loss: 0.0000, D(x): 1.00, D(G(z)): 0.00
Saving pre-train_images-0002.png
Epoch [2/50], Step [200/600], d_loss: 0.0000, g_loss: 0.0000, D(x): 1.00, D(G(z)): 0.00
Epoch [2/50], Step [400/600], d_loss: 0.0000, g_loss: 0.0000, D(x): 1.00, D(G(z)): 0.00
Epoch [2/50], Step [600/600], d_loss: 0.0000, g_loss: 0.0000, D(x): 1.00, D(G(z)): 0.00
Saving pre-train_images-0003.png
Epoch [3/50], Step [200/600], d_loss: 0.0000, g_loss: 0.0000, D(x): 1.00, D(G(z)): 0.00
Epoch [3/50], Step [4

Epoch [27/50], Step [600/600], d_loss: 0.0000, g_loss: 0.0000, D(x): 1.00, D(G(z)): 0.00
Saving pre-train_images-0028.png
Epoch [28/50], Step [200/600], d_loss: 0.0000, g_loss: 0.0000, D(x): 1.00, D(G(z)): 0.00
Epoch [28/50], Step [400/600], d_loss: 0.0000, g_loss: 0.0000, D(x): 1.00, D(G(z)): 0.00
Epoch [28/50], Step [600/600], d_loss: 0.0000, g_loss: 0.0000, D(x): 1.00, D(G(z)): 0.00
Saving pre-train_images-0029.png
Epoch [29/50], Step [200/600], d_loss: 0.0000, g_loss: 0.0000, D(x): 1.00, D(G(z)): 0.00
Epoch [29/50], Step [400/600], d_loss: 0.0000, g_loss: 0.0000, D(x): 1.00, D(G(z)): 0.00
Epoch [29/50], Step [600/600], d_loss: 0.0000, g_loss: 0.0000, D(x): 1.00, D(G(z)): 0.00
Saving pre-train_images-0030.png
Epoch [30/50], Step [200/600], d_loss: 0.0000, g_loss: 0.0000, D(x): 1.00, D(G(z)): 0.00
Epoch [30/50], Step [400/600], d_loss: 0.0000, g_loss: 0.0000, D(x): 1.00, D(G(z)): 0.00
Epoch [30/50], Step [600/600], d_loss: 0.0000, g_loss: 0.0000, D(x): 1.00, D(G(z)): 0.00
Saving pre-

In [32]:
import cv2
import os
from IPython.display import FileLink

vid_fname = sample_dir + '/gans_training.avi'

files = [os.path.join(sample_dir, f) for f in os.listdir(sample_dir) if 'fake_images' in f]
files.sort()

out = cv2.VideoWriter(vid_fname,cv2.VideoWriter_fourcc(*'MP4V'), 8, (302,302))
[out.write(cv2.imread(fname)) for fname in files]
out.release()
FileLink(vid_fname)

/workspace/fl/data/samples/gans_training.avi

In [ ]:
plt.plot(d_losses, '-')
plt.plot(g_losses, '-')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['Discriminator', 'Generator'])
plt.title('Losses');